# Example of Sensitivity Analysis

In [ ]:
# Defaults
n_samples = 1024 * 21

In [ ]:
from itertools import repeat

import numpy as np
import pandas as pd

import dask.bag as db
from dask.distributed import Client
from dask_jobqueue import SLURMCluster as Cluster

from SALib.sample import latin
from SALib.analyze import delta as delta_mim

from mhpc_project.matsch_b2 import Variables, CalibrationModel, Loss
from geotopy.measures import KGE
from geotopy.utils import date_parser

In [ ]:
observations = pd.read_csv('../data/Matsch B2/obs.csv',
                           na_values=['-9999', '-99.99'],
                           usecols=[0, 7],
                           parse_dates=[0],
                           date_parser=date_parser,
                           index_col=0,
                           squeeze=True)
observations.index.rename('datetime', inplace=True)
model = CalibrationModel('../data/Matsch B2/geotop', run_args={'timeout': 120})
variables = Variables('../data/Matsch B2/variables.csv')
measure = KGE(observations)
loss = Loss(model, variables, measure)

In [ ]:
cluster = Cluster()
client = Client(cluster)

In [ ]:
problem = {'num_vars': variables.num_vars,
           'names': variables.names,
           'bounds': list(repeat((0.0, 1.0), variables.num_vars))}

samples = latin.sample(problem, n_samples)

In [ ]:
while remaining_budget or self._running_jobs or self._finished_jobs:
            # # # # # Update optimizer with finished jobs # # # # #
            # this is the first thing to do when resuming an existing optimization run
            # process finished
            if self._finished_jobs:
                if (remaining_budget or sleeper._start is not None) and not first_iteration:
                    # ignore stop if no more suggestion is sent
                    # this is an ugly hack to avoid warnings at the end of steady mode
                    sleeper.stop_timer()
                while self._finished_jobs:
                    x, job = self._finished_jobs[0]
                    result = job.result()
                    if multiobjective:  # hack
                        result = objective_function.compute_aggregate_loss(job.result(), *x.args, **x.kwargs)  # type: ignore
                    self.tell(x, result)
                    self._finished_jobs.popleft()  # remove it after the tell to make sure it was indeed "told" (in case of interruption)
                    if verbosity:
                        print(f"Updating fitness with value {job.result()}")
                if verbosity:
                    print(f"{remaining_budget} remaining budget and {len(self._running_jobs)} running jobs")
                    if verbosity > 1:
                        print("Current pessimistic best is: {}".format(self.current_bests["pessimistic"]))
            elif not first_iteration:
                sleeper.sleep()
            # # # # # Start new jobs # # # # #
            if not batch_mode or not self._running_jobs:
                new_sugg = max(0, min(remaining_budget, self.num_workers - len(self._running_jobs)))
                if verbosity and new_sugg:
                    print(f"Launching {new_sugg} jobs with new suggestions")
                for _ in range(new_sugg):
                    args = self.ask()
                    self._running_jobs.append((args, executor.submit(func, *args.args, **args.kwargs)))
                if new_sugg:
                    sleeper.start_timer()
            remaining_budget = self.budget - self.num_ask
            # split (repopulate finished and runnings in only one loop to avoid
            # weird effects if job finishes in between two list comprehensions)
            tmp_runnings, tmp_finished = [], deque()
            for x_job in self._running_jobs:
                (tmp_finished if x_job[1].done() else tmp_runnings).append(x_job)
            self._running_jobs, self._finished_jobs = tmp_runnings, tmp_finished
            first_iteration = False

In [ ]:
bag = db.from_sequence(samples, npartitions=512).map(loss)
losses = bag.compute()

In [ ]:
losses = np.array(losses)
indices = np.isfinite(losses)
good_samples = samples[indices]
good_losses = losses[indices]

SA = delta_mim.analyze(problem, good_samples, good_losses)

In [ ]:
SA.to_df().sort_values('delta', key=np.abs, ascending=False)